In [14]:
import os
import sys

from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import SimpleITK as sitk
import nrrd
import vtk

import torch
from torch import nn
from torch.nn import functional as F
from torchvision import transforms

import pytorch_lightning as pl
import pickle
import monai 
import glob 
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

sys.path.append('/mnt/famli_netapp_shared/C1_ML_Analysis/src/famli-ultra-sim/')
sys.path.append('/mnt/famli_netapp_shared/C1_ML_Analysis/src/famli-ultra-sim/dl')
import dl.loaders.ultrasound_dataset as ultrasound_dataset
import dl.transforms.ultrasound_transforms as ultrasound_transforms
import dl.loaders.mr_us_dataset as mr_us_dataset
import dl.nets.us_simulation_jit as us_simulation_jit
import dl.nets.us_simu as us_simu

import importlib

from dl.nets.layers import TimeDistributed

from torchvision import transforms as T
from torchvision import models as tv_models
from torchvision.ops import Conv2dNormActivation

sys.path.append('/mnt/raid/C1_ML_Analysis/source/ShapeAXI/src/')

from shapeaxi.saxi_layers import AttentionChunk, SelfAttention


In [2]:
model = tv_models.efficientnet_v2_s().features


In [3]:
model[0]

In [8]:
model[0] = Conv2dNormActivation(1, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False, norm_layer=nn.BatchNorm2d, activation_layer=nn.SiLU)
model[0]

In [6]:
f = model(torch.rand(1, 1, 256, 256))
f.shape

In [24]:
class AttentionChunk(nn.Module):
    def __init__(self, input_dim, hidden_dim, chunks=8):
        super().__init__()
        
        self.chunks = chunks
        self.attn = SelfAttention(input_dim, hidden_dim)

    def forward(self, x):

        # Shape of x is [BS, T, C, H, W] or [BS, C, T, H, W]
        # assert len(x.shape) == 5

        x_out = []
        x_shape = list(x.shape)
        x_shape[1] = self.chunks
        
        for ch in torch.chunk(x, chunks=self.chunks, dim=1): # Iterate in the time dimension and create chunks
            ch, _ = self.attn(ch, ch)
            x_out.append(ch)
        x_out = torch.stack(x_out, dim=1)
        
        return x_out

In [27]:
attn = AttentionChunk(1280, 64, chunks=8)
out = attn(torch.rand(1, 30, 8, 8, 1280))